# Задание по теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

In [1]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV, cross_validate
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np

Создадим датафрейм с форматом пригодным для библиотеки Surprise. Для первичного подбора параметров загрузим датасет на 100К.

In [2]:
df_movies = pd.read_csv('movielens_100k\\movies.csv')
df_ratings = pd.read_csv('movielens_100k\\ratings.csv')
df_ratings = df_ratings.merge(df_movies.loc[:, ['movieId', 'title']], 
                              on='movieId', how='left').loc[:, ['userId', 'title', 'rating']]
df_ratings[:3]

,userId,title,rating
0,1,Toy Story (1995),4.0
1,1,Grumpier Old Men (1995),4.0
2,1,Heat (1995),4.0


Найдем минимальную и максимальную оценки для калибровки объекта класса Reader.

In [3]:
rating_min, rating_max = df_ratings.rating.min(), df_ratings.rating.max()
print("Минимальный рейтинг: {}\nМаксимальный рейтинг: {}".format(rating_min, rating_max))

Минимальный рейтинг: 0.5
Максимальный рейтинг: 5.0


Создадим датасет в формате библиотеки Surprise

In [4]:
reader = Reader(rating_scale=(rating_min, rating_max))
data = Dataset.load_from_df(df_ratings, reader)

Попробуем побучить модель SVD и предсказать оценку пользователя к каждому фильму

In [5]:
model_svd = SVD()
_ = cross_validate(model_svd, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8832  0.8667  0.8724  0.8720  0.8751  0.8739  0.0054  
MAE (testset)     0.6794  0.6653  0.6707  0.6691  0.6723  0.6713  0.0046  
Fit time          7.76    7.57    10.01   8.13    6.30    7.95    1.20    
Test time         0.24    0.21    0.29    0.21    0.21    0.23    0.03    


In [6]:
"""
!!! Блок выполняется около 1-2 часов
"""
param_grid = {
    'n_factors': [10, 25, 50, 75, 100, 150, 200],
    'n_epochs': [20, 40, 60],
    'lr_all': [0.005, 0.01, 0.05]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8623182447822622
{'n_factors': 200, 'n_epochs': 40, 'lr_all': 0.05}


Поиск наилучших параметров с помощью решетчатого поиска дал следующие результаты:

- Количество факторов 200
- Количество эпох 40
- Скорость обучения для всех параметров SVD 0.05

Данные параметры дают RMSE 0.8623 на датасете 100K.

Попробуем обучить модель с этими параметрами на датасете 1М.

In [7]:
df_movies = pd.read_csv('movielens_1m\\movies.dat', delimiter="::", 
                        names=['movieId', 'title', 'genres'], engine='python')
df_ratings = pd.read_csv('movielens_1m\\ratings.dat', delimiter="::", 
                         names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')
df_ratings = df_ratings.merge(df_movies.loc[:, ['movieId', 'title']], 
                              on='movieId', how='left').loc[:, ['userId', 'title', 'rating']]
df_ratings[:3]

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3


In [8]:
model_svd = SVD(n_factors=200, n_epochs=40, lr_all=0.05)
crosvall_result = cross_validate(model_svd, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8576  0.8589  0.8693  0.8638  0.8645  0.8628  0.0042  
MAE (testset)     0.6570  0.6556  0.6642  0.6632  0.6623  0.6605  0.0035  
Fit time          27.79   20.70   20.11   19.98   19.87   21.69   3.06    
Test time         0.15    0.16    0.16    0.23    0.16    0.17    0.03    


Построим простую рекомендательную систему, которая найдет ТОП-10 похожих фильмов с самой высокой оценкой для пользователя с идентификатором 414.

In [9]:
def genre_prepare(genre):
    genre = ' '.join(genre.replace('-', '').replace(' ', '').lower().split('|'))
    return genre

def get_user_rating(user_id, title):
    """ Поиск оценки пользователя для фильма в датасете MovieLens
    
    Параметры
    ---------
    user_id : int
      Идентификатор пользователя
    title : str
      Название фильма
    """
    user_id = int(user_id)
    user_rating = list(df_ratings.loc[(df_ratings.userId == user_id) &
                                      (df_ratings.title == title), 'rating'].values)
    user_rating = user_rating[0] if user_rating else None
    return user_rating

In [10]:
df_movies['genres_mod'] = df_movies['genres'].apply(genre_prepare)
df_movies['user_rating'] = df_movies.apply(lambda x: get_user_rating(user_id=414, title=x.title), axis=1)
df_movies[:5]

,movieId,title,genres,genres_mod,user_rating
0,1,Toy Story (1995),Animation|Children's|Comedy,animation children's comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,adventure children's fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,comedy romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,comedy drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,comedy,NaN


Создадим датафрейм с фильмами, у которых нет оценки пользователя. То есть эти фильмы предположительно пользователь не смотрел. У нас нет информации о контакте пользователя с контентом без выставления ему оценки, поэтому делаем такое упрощенное предположение.

In [11]:
df_no_watched_user_movies = df_movies[df_movies.user_rating.isna()]
df_no_watched_user_movies[:5]

,movieId,title,genres,genres_mod,user_rating
0,1,Toy Story (1995),Animation|Children's|Comedy,animation children's comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,adventure children's fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,comedy romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,comedy drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,comedy,NaN


Создаем матрицу TF-IDF для жанров непросмотренных фильмов 

In [12]:
tfidf = TfidfVectorizer()
no_watched_user_movies_tfidf = tfidf.fit_transform(df_no_watched_user_movies['genres_mod']).todense()

Предположим, что пользователь посмотрел фильм "Requiem for a Dream (2000)" и требуется предложить ТОП-10 похожих фильмов с максимальной предсказанной пользователськой оценкой. И эти фильмы не должны быть просмотрены пользователем.

In [13]:
# Находим жанр просмотренного фильма
watсhed_film_genre = df_movies.loc[df_movies.title == "Requiem for a Dream (2000)", 'genres_mod']
# Находим матрицу TF-IDF для просмотренного фильма 
watсhed_film_genre_tfidf = tfidf.transform(watсhed_film_genre).todense()

# Расчитаем косинусное сходство между просмотренным фильмом и остальными непросмотренными
cosine_result = cosine_similarity(no_watched_user_movies_tfidf, watсhed_film_genre_tfidf).flatten()

# Находим 500 индексов фильмов, которые наибольшим образом похожи на просмотренный фильм
top_film_index = np.argsort(cosine_result)[::-1][:500]

# Для 500 наиболее похожих фильмов находим рейтинг пользователя и выдаем ТОП-10 с наибольшей оценкой пользователя
df_user_movies_recommend = df_no_watched_user_movies.iloc[top_film_index, :].copy()
df_user_movies_recommend['user_rating'] = df_user_movies_recommend['title'].apply(lambda x: model_svd.predict(uid='414', iid=x).est)
df_user_movies_recommend.sort_values(by='user_rating', ascending=False)[:10]

,movieId,title,genres,genres_mod,user_rating
315,318,"Shawshank Redemption, The (1994)",Drama,drama,4.249196
1028,1041,Secrets & Lies (1996),Drama,drama,4.247093
3399,3468,"Hustler, The (1961)",Drama,drama,4.224298
3132,3201,Five Easy Pieces (1970),Drama,drama,4.188433
3278,3347,Never Cry Wolf (1983),Drama,drama,4.150602
2291,2360,"Celebration, The (Festen) (1998)",Drama,drama,4.133238
942,954,Mr. Smith Goes to Washington (1939),Drama,drama,4.131595
2890,2959,Fight Club (1999),Drama,drama,4.098818
2260,2329,American History X (1998),Drama,drama,4.090155
3155,3224,Woman in the Dunes (Suna no onna) (1964),Drama,drama,4.080022
